In [1]:
# The case study is from an open source dataset from Kaggle. 

# Link to the Kaggle project site:

# https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling

# Given a Bank customer, can we build a classifier which can determine whether they will leave or not using Neural networks?

In [2]:
# importing required libraries
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
tf.random.set_seed(42)
import numpy as np
import matplotlib.pyplot as plt
from keras import optimizers
tf.__version__

Using TensorFlow backend.


'2.0.0'

In [3]:
#1. Read the dataset
df = pd.read_csv("Churn.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()
# There are 3 features which ate of object data type which we will be converting to numeric in further steps.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.describe()
# There are no missing values.

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
df['Exited'].value_counts()

# Observing a class imbalance. We have more data points for customers who have not exited than who have exited. 

0    7963
1    2037
Name: Exited, dtype: int64

In [7]:
#2. Drop the columns which are unique for all users like IDs (2.5 points)

#Dropping columns ID and Rownumber, Surname

df = df.drop(columns = ['RowNumber','CustomerId','Surname'], axis = 1)
df.shape

(10000, 11)

In [20]:
# Converting Categorical variable to numeric
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
le= LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['Geography'] = le.fit_transform(df['Geography'])


In [21]:
#3. Distinguish the feature and target set (2.5 points)
X = df.drop(columns = 'Exited', axis = 1)
Y = df['Exited']

In [22]:
#4. Divide the data set into Train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=1, stratify = Y)
X_train.shape

(8000, 10)

In [23]:
#5. Normalize the train and test data (2.5 points)
# normalising using standard scalar
from sklearn.preprocessing import StandardScaler
X_train_std = StandardScaler().fit_transform(X_train)
X_test_std = StandardScaler().fit_transform(X_test)
Y_train = Y_train.to_numpy()
Y_test = Y_test.to_numpy()

C:\Users\Aadee\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Aadee\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Aadee\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Aadee\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [25]:
#6. Initialize & build the model (7.5 points)
#Initialize Sequential Graph (model)

from keras.models import Sequential
from keras.layers import Dense

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(activation = 'relu', input_dim = 10, units=6))
model.add(tf.keras.layers.Dense(20, activation = 'relu')) 
model.add(tf.keras.layers.Dense(1,activation = 'sigmoid')) 
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(X_train_std, Y_train, validation_data=(X_test_std,Y_test),epochs=100,batch_size=32)

# This model is a good fit as it has both train & test accuracy around 86%. 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_4 (Dense)              (None, 20)                140       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 21        
Total params: 227
Trainable params: 227
Non-trainable params: 0
_________________________________________________________________
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 1s 100us/sample - loss: 0.5498 - accuracy: 0.7853 - val_loss: 0.5109 - val_accuracy: 0.7965
Epoch 2/100
8000/8000 [==============================] - 0s 48us/sample - loss: 0.4972 - accuracy: 0.7962 - val_loss: 0.4857 - val_accuracy: 0.7965
Epoch 3/100
8000/8000 [==================

Epoch 51/100
8000/8000 [==============================] - 0s 44us/sample - loss: 0.3439 - accuracy: 0.8604 - val_loss: 0.3539 - val_accuracy: 0.8645
Epoch 52/100
8000/8000 [==============================] - 0s 44us/sample - loss: 0.3436 - accuracy: 0.8590 - val_loss: 0.3539 - val_accuracy: 0.8625
Epoch 53/100
8000/8000 [==============================] - 0s 44us/sample - loss: 0.3432 - accuracy: 0.8580 - val_loss: 0.3540 - val_accuracy: 0.8645
Epoch 54/100
8000/8000 [==============================] - 0s 47us/sample - loss: 0.3432 - accuracy: 0.8586 - val_loss: 0.3538 - val_accuracy: 0.8655
Epoch 55/100
8000/8000 [==============================] - 0s 45us/sample - loss: 0.3429 - accuracy: 0.8591 - val_loss: 0.3536 - val_accuracy: 0.8645
Epoch 56/100
8000/8000 [==============================] - 0s 47us/sample - loss: 0.3427 - accuracy: 0.8586 - val_loss: 0.3541 - val_accuracy: 0.8665
Epoch 57/100
8000/8000 [==============================] - 0s 45us/sample - loss: 0.3425 - accuracy: 0.8585

In [26]:
#7. Optimize the model (5 points)
from keras import optimizers
sgd = tf.keras.optimizers.SGD(lr=0.001)  # With learning rate 
#Compile the model
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(X_train_std, Y_train, validation_data=(X_test_std,Y_test),epochs=100,batch_size=32)

# Even after optimising with a reduced learning rate, both train & test accuracy is around 86%

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_4 (Dense)              (None, 20)                140       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 21        
Total params: 227
Trainable params: 227
Non-trainable params: 0
_________________________________________________________________
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 1s 92us/sample - loss: 0.3356 - accuracy: 0.8589 - val_loss: 0.3486 - val_accuracy: 0.8640
Epoch 2/100
8000/8000 [==============================] - 0s 46us/sample - loss: 0.3354 - accuracy: 0.8595 - val_loss: 0.3486 - val_accuracy: 0.8635
Epoch 3/100
8000/8000 [===================

Epoch 51/100
8000/8000 [==============================] - 0s 56us/sample - loss: 0.3348 - accuracy: 0.8610 - val_loss: 0.3484 - val_accuracy: 0.8640
Epoch 52/100
8000/8000 [==============================] - 0s 56us/sample - loss: 0.3348 - accuracy: 0.8605 - val_loss: 0.3484 - val_accuracy: 0.8645
Epoch 53/100
8000/8000 [==============================] - 0s 57us/sample - loss: 0.3348 - accuracy: 0.8608 - val_loss: 0.3483 - val_accuracy: 0.8650
Epoch 54/100
8000/8000 [==============================] - 0s 55us/sample - loss: 0.3348 - accuracy: 0.8602 - val_loss: 0.3484 - val_accuracy: 0.8640
Epoch 55/100
8000/8000 [==============================] - 0s 54us/sample - loss: 0.3348 - accuracy: 0.8604 - val_loss: 0.3484 - val_accuracy: 0.8645
Epoch 56/100
8000/8000 [==============================] - 0s 56us/sample - loss: 0.3348 - accuracy: 0.8604 - val_loss: 0.3483 - val_accuracy: 0.8645
Epoch 57/100
8000/8000 [==============================] - 0s 54us/sample - loss: 0.3348 - accuracy: 0.8602

In [33]:
#9. Predict the results using 0.5 as a threshold (5 points)
y_pred = model.predict(X_test_std)
y_pred = (y_pred > 0.5)
y_pred.shape

(2000, 1)

In [34]:
#10.Print the Accuracy score and confusion matrix (5 points)
from sklearn import metrics
test_loss, test_acc = model.evaluate(x=X_test_std,y=Y_test)
train_loss, train_acc = model.evaluate(x=X_train_std,y=Y_train)
print("training accuracy:", train_acc)
print("training loss:", train_loss)
print("test accuracy:", test_acc)
print("test loss:", test_loss)

conf_mat = metrics.confusion_matrix(y_true=Y_test, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)
print(metrics.classification_report(Y_test, y_pred))

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

8000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

training accuracy: 0.860375
training loss: 0.3343153082728386
test accuracy: 0.8635
test loss: 0.3480367648601532
Confusion matrix:
 [[1540   53]
 [ 220  187]]
              precision    recall  f1-score   support

           0       0.88      0.97      0.92      1593
           1       0.78      0.46      0.58       407

   micro avg       0.86      0.86      0.86      2000
   macro avg       0.83      0.71      0.75      2000
weighted avg       0.86      0.86      0.85      2000



In [ ]:
# Even though we are seeing a good train & test accuracy,by looking at confusion matrix, recall & precision rate for exited /exiting
# customers is not so good. Howewver, for continuing cutomers precision & recall is rate is good. Since we had a imbalanced data set, 
# it was expected to provide better results for class 0 / continuing cutomers class. 